# Cache Assignment

In [15]:
!lscpu

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          39 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   16
  On-line CPU(s) list:    0-15
Vendor ID:                GenuineIntel
  Model name:             12th Gen Intel(R) Core(TM) i5-12500H
    CPU family:           6
    Model:                154
    Thread(s) per core:   2
    Core(s) per socket:   8
    Socket(s):            1
    Stepping:             3
    BogoMIPS:             6220.80
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht sysc
                          all nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xt
                          opology tsc_reliable nonstop_tsc cpuid pni pclmulqdq v
                          mx ssse3 fma cx16 pcid sse4_1 sse4_2

In [16]:
import numpy as np

class Cache:
    def __init__(self, BLOCK_SIZE, CACHE_SIZE, associativity):
        self.BLOCK_SIZE = BLOCK_SIZE
        self.associativity = associativity
        self.num_sets = CACHE_SIZE // (BLOCK_SIZE * associativity)

        # Initialize the cache data: valid bits and tags
        self.valid_bits = np.zeros((self.num_sets, associativity), dtype=np.int8)
        self.tags = np.zeros((self.num_sets, associativity), dtype=np.int32)
        self.lru = np.zeros((self.num_sets, associativity), dtype=np.int8)  # To track LRU

    def access(self, address):
        # Calculate the cache index and tag
        line = address // self.BLOCK_SIZE
        cache_index = line % self.num_sets
        tag = line // self.num_sets

        # Check if the tag is in the cache set
        for way in range(self.associativity):
            if self.valid_bits[cache_index, way] == 1 and self.tags[cache_index, way] == tag:
                # Cache hit
                self.update_lru(cache_index, way)
                return True

        # Cache miss
        self.replace_line(cache_index, tag)
        return False

    def update_lru(self, cache_index, way):
        # Update the LRU status for all ways in the set
        self.lru[cache_index] = np.where(self.lru[cache_index] > self.lru[cache_index, way],
                                         self.lru[cache_index] - 1,
                                         self.lru[cache_index])
        self.lru[cache_index, way] = self.associativity - 1

    def replace_line(self, cache_index, tag):
        # Find the least recently used (LRU) way to replace
        lru_way = np.argmin(self.lru[cache_index])
        self.tags[cache_index, lru_way] = tag
        self.valid_bits[cache_index, lru_way] = 1
        self.update_lru(cache_index, lru_way)

# Function to determine cache layout description
def describe_cache_layout(associativity, num_sets):
    if associativity == 1:
        return "Direct-Mapped Cache"
    elif num_sets == 1:
        return "Fully-Associative Cache"
    else:
        return f"{associativity}-Way Set-Associative Cache"

# Function to simulate cache accesses and calculate miss rate
def simulate_cache(array_size, element_size, base_address, BLOCK_SIZE, CACHE_SIZE, associativity):
    cache = Cache(BLOCK_SIZE, CACHE_SIZE, associativity)

    row_major_hits, column_major_hits = 0, 0

    # Row-major access
    for i in range(array_size):
        for j in range(array_size):
            address = base_address + (i * array_size + j) * element_size
            if cache.access(address):
                row_major_hits += 1

    row_major_miss_rate = 1 - (row_major_hits / (array_size * array_size))

    # Reset cache for column-major test
    cache = Cache(BLOCK_SIZE, CACHE_SIZE, associativity)

    # Column-major access
    for j in range(array_size):
        for i in range(array_size):
            address = base_address + (i * array_size + j) * element_size
            if cache.access(address):
                column_major_hits += 1

    column_major_miss_rate = 1 - (column_major_hits / (array_size * array_size))

    return row_major_miss_rate, column_major_miss_rate

def calculate_array_size(CACHE_SIZE, element_size):
    total_elements = CACHE_SIZE // element_size  # Total elements that fit in the cache
    array_size = int(total_elements**0.5)       # Determine the size of the square array
    return array_size

# Constants
BLOCK_SIZE = 64  # Bytes
CACHE_SIZE = 4096  # Bytes
L3_CACHE_SIZE = 18 * 1024 * 1024  # Bytes
associativity = 4

base_address = 0x8aa1000  # Start at this arbitrary address


print("Question 1:")
# Cache layout description
cache_layout = describe_cache_layout(associativity, CACHE_SIZE // (BLOCK_SIZE * associativity))
print(f"Cache Layout: {cache_layout}\n")

# Data types and configurations
data_types = [
    {"name": "char", "element_size": 1, "array_size": 64},
    {"name": "short", "element_size": 2, "array_size": calculate_array_size(L3_CACHE_SIZE, 2)},
    {"name": "int", "element_size": 4, "array_size": 32},
    {"name": "long", "element_size": 8, "array_size": calculate_array_size(L3_CACHE_SIZE, 8)},
]

# Simulate and report miss rates
print("Question 2:")
# Header
print(f"{'Data Type':<12} | {'Element Size':<12} | {'Array Size':<15} | {'Row-Major Miss Rate':<22} | {'Column-Major Miss Rate':<22}")
print("-" * 95)

for data_type in data_types:
    row_miss, col_miss = simulate_cache(
        data_type["array_size"],
        data_type["element_size"],
        base_address,
        BLOCK_SIZE,
        CACHE_SIZE,
        associativity
    )
    array_size_str = f"{data_type['array_size']}x{data_type['array_size']}"
    print(f"{data_type['name']:<12} | {data_type['element_size']:<12} | {array_size_str:<15} | {row_miss * 100:>6.2f}%{'':<15} | {col_miss * 100:>6.2f}%{'':<15}")


Question 1:
Cache Layout: 4-Way Set-Associative Cache

Question 2:
Data Type    | Element Size | Array Size      | Row-Major Miss Rate    | Column-Major Miss Rate
-----------------------------------------------------------------------------------------------
char         | 1            | 64x64           |   1.56%                |   1.56%               
short        | 2            | 3072x3072       |   3.12%                | 100.00%               
int          | 4            | 32x32           |   6.25%                |   6.25%               
long         | 8            | 1536x1536       |  12.50%                | 100.00%               


In [ ]:
import random

# Function to simulate random cache accesses and calculate miss rate
def simulate_random_access(element_size, base_address, BLOCK_SIZE, CACHE_SIZE, associativity, L3_CACHE_SIZE):
    cache = Cache(BLOCK_SIZE, CACHE_SIZE, associativity)
    random_hits = 0
    num_accesses = L3_CACHE_SIZE // element_size // 2  # Half the range of L3_CACHE_SIZE

    # Perform random accesses
    for _ in range(num_accesses):
        address = base_address + random.randint(0, L3_CACHE_SIZE // 2) * element_size
        if cache.access(address):
            random_hits += 1

    random_miss_rate = 1 - (random_hits / num_accesses)
    return random_miss_rate

# Header for random accesses
print("\nQuestion 3: Random Access Miss Rates")
print(f"{'Data Type':<12} | {'Element Size':<12} | {'Array Size':<15} | {'Random Access Miss Rate':<30}")
print("-" * 80)

for data_type in data_types:
    random_miss = simulate_random_access(
        data_type["element_size"],
        base_address,
        BLOCK_SIZE,
        CACHE_SIZE,
        associativity,
        L3_CACHE_SIZE
    )
    array_size_str = f"{data_type['array_size']}x{data_type['array_size']}"
    print(f"{data_type['name']:<12} | {data_type['element_size']:<12} | {array_size_str:<15} | {random_miss * 100:>6.2f}%{'':<15}")





Question 3: Random Access Miss Rates
Data Type    | Element Size | Array Size      | Random Access Miss Rate       
--------------------------------------------------------------------------------



# Key Observations:
### Row-Major:
 Sequential data access leads to high cache efficiency due to effective block prefetching.
### Column-Major:
Larger data types (e.g., short and long) experience 100% miss rates due to poor spatial locality. Each access spans different cache blocks, leading to frequent evictions.
### Random Access:
 Miss rates are near 100% for all data types, regardless of size. Random accesses destroy any spatial or temporal locality, rendering it nearly impossible for the cache to reuse or prefetch data.
## Brief Summary
### Sequential Access:
The cache performs well for row-major patterns with small data types due to efficient block prefetching. However, column-major patterns show significant inefficiency for larger data types.
### Random Access:
The cache is very ineffective for random access patterns, with nearly 100% miss rates across all data types and sizes. This demonstrates how important memory access patterns are in performance optimization.
### Data Type Impact:
Larger data types amplify cache inefficiencies in both sequential and random patterns due to their larger memory footprints and block misalignment.